# IBM Data Science Professional Certificate - Course 9 Captsone Assignment

## Alex Yu

This notebook will be used thorughout the course for the capstone assignment

In [2]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 
Below cell reads in the scraped csv file containing the columns of Neighborhood, Longitude, and Latitude of New Orleans

In [3]:
NO_start = pd.read_csv("NewOrleans.csv")
NO_start.head()

,Neighborhood,Longitude,Latitude
0,U.S. NAVAL BASE,-90.026093,29.946085
1,ALGIERS POINT,-90.051606,29.952462
2,WHITNEY,-90.042357,29.947200
3,AUDUBON,-90.121450,29.932994
4,OLD AURORA,-90.000000,29.924440


## 

Using folium create a map and plotting markers with corresponding coordinates labelled the Neighborhood values

In [4]:
NO_lat = 29.9511
NO_long = -90.0715
# create map of New Orleans using latitude and longitude values from Google
map_NO = folium.Map(location=[NO_lat, NO_long], zoom_start=11)
# add markers to map
for lat, lng, neighborhood in zip(NO_start['Latitude'], NO_start['Longitude'], NO_start['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NO)  
    
map_NO

## 
Below, estabalish credentials and fields for Foursquare

In [5]:
CLIENT_ID = 'W5NDD2HHSO4FRCFRFLEGZFM0WHXT3GX3ZYJWTTYEV0DEAI5I' # my Foursquare ID
CLIENT_SECRET = '4EZCAKXDK04KX5TTPNN5MBOMNF21KEKKG4E2MVWHXSOFHDTY' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

## 
Method that iteratively acquires all the venues nearby for all the neighborhoods in the dataset

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## 
Create a dataframe with all the venues retrieved from Foursquare API

In [7]:
NO_venues = getNearbyVenues(names=NO_start['Neighborhood'],
                                   latitudes=NO_start['Latitude'],
                                   longitudes=NO_start['Longitude']
                                  )

U.S. NAVAL BASE
ALGIERS POINT
WHITNEY
AUDUBON
OLD AURORA
B. W. COOPER
BAYOU ST. JOHN
BEHRMAN
BLACK PEARL
BROADMOOR
MARLYVILLE - FONTAINEBLEAU
GERT TOWN
MID-CITY
ST. CLAUDE
CENTRAL BUSINESS DISTRICT
FRENCH QUARTER
CENTRAL CITY
LAKE CATHERINE
VILLAGE DE LEST
VIAVANT - VENETIAN ISLES
NEW AURORA - ENGLISH TURN
TALL TIMBERS - BRECHTEL
FISCHER DEV
McDONOGH
LOWER GARDEN DISTRICT
ST. THOMAS DEV
EAST RIVERSIDE
IRISH CHANNEL
TOURO
MILAN
UPTOWN
WEST RIVERSIDE
EAST CARROLLTON
FRERET
GARDEN DISTRICT
LEONIDAS
HOLLYGROVE
TULANE - GRAVIER
TREME - LAFITTE
SEVENTH WARD
MARIGNY
ST. ROCH
DIXON
LAKEWOOD
NAVARRE
CITY PARK
LAKEVIEW
WEST END
LAKESHORE - LAKE VISTA
FILMORE
ST. BERNARD AREA
DILLARD
ST. ANTHONY
LAKE TERRACE & OAKS
MILNEBURG
PONTCHARTRAIN PARK
GENTILLY WOODS
GENTILLY TERRACE
DESIRE AREA
FLORIDA AREA
FLORIDA DEV
LOWER NINTH WARD
BYWATER
HOLY CROSS
PINES VILLAGE
PLUM ORCHARD
READ BLVD WEST
READ BLVD EAST
WEST LAKE FOREST
LITTLE WOODS
FAIRGROUNDS
IBERVILLE


In [8]:
print(NO_venues.shape)
NO_venues.head()

(907, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,U.S. NAVAL BASE,29.946085,-90.026093,The Mighty Missisippi,29.949695,-90.023710,Boat or Ferry
1,ALGIERS POINT,29.952462,-90.051606,Tout de Suite Café,29.952121,-90.051090,Café
2,ALGIERS POINT,29.952462,-90.051606,Congregation Coffee Roasters,29.951918,-90.053395,Coffee Shop
3,ALGIERS POINT,29.952462,-90.051606,Levee @ Algiers Point,29.951760,-90.048747,Scenic Lookout
4,ALGIERS POINT,29.952462,-90.051606,The Crown & Anchor,29.951416,-90.054220,Bar


## 
Group the venues dataframe by Neighborhood

In [9]:
NO_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALGIERS POINT,23,23,23,23,23,23
AUDUBON,12,12,12,12,12,12
B. W. COOPER,4,4,4,4,4,4
BAYOU ST. JOHN,7,7,7,7,7,7
BLACK PEARL,4,4,4,4,4,4
...,...,...,...,...,...,...
UPTOWN,24,24,24,24,24,24
WEST END,2,2,2,2,2,2
WEST LAKE FOREST,9,9,9,9,9,9


## 
Figure out how many unique venues there are

In [10]:
print('There are {} uniques categories.'.format(len(NO_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [11]:
# one hot encoding
NO_onehot = pd.get_dummies(NO_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NO_onehot['Neighborhood'] = NO_venues['Neighborhood'] 

# move neighborhood column to the first column
cols_to_move = ['Neighborhood', 'Accessories Store']
NO_onehot = NO_onehot[ cols_to_move + [ col for col in NO_onehot.columns if col not in cols_to_move ] ]

NO_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vehicle Inspection Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,U.S. NAVAL BASE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALGIERS POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 
Create a new dataframe that takes the mean of frequencies from our above dataframe object

In [76]:
NO_grouped = NO_onehot.groupby('Neighborhood').mean().reset_index()
NO_grouped.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vehicle Inspection Station,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,ALGIERS POINT,0.0,0.0,0.0,0.000000,0.0,0.043478,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0
1,AUDUBON,0.0,0.0,0.0,0.083333,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,B. W. COOPER,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,BAYOU ST. JOHN,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,BLACK PEARL,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


## 
Limit the scope of the venues to those of restaurants/related to food or hospitality or tourism

In [89]:
venue_list = ['Neighborhood','American Restaurant','Asian Restaurant', 'BBQ Joint', 'Bagel Shop', 'Bakery', 'Bar', 
              'Baseball Field', 'Baseball Stadium', 'Basketball Court', 'Bed & Breakfast','Beer Garden', 'Beer Store', 'Bistro', 'Breakfast Spot', 'Brewery', 'Burger Joint', 'Burrito Place',
             'Café', 'Cajun / Creole Restaurant', 'Candy Store', 'Caribbean Restaurant', 'Cheese Shop', 'Chinese Restaurant', 'Cocktail Bar', 'Coffee Shop', 'Comedy Club', 'Comfort Food Restaurant', 'Concert Hall',
             'Deli / Bodega', 'Dessert Shop', 'Dive Bar', 'Donut Shop', 'Eastern European Restaurant', 'Ethiopian Restaurant', 'Farmers Market', 'Fast Food Restaurant', 'Flea Market', 'Flower Shop', 
              'Food', 'Food & Drink Shop', 'Food Court', 'Food Truck', 'French Restaurant', 'Fried Chicken Joint', 'Gay Bar', 'General Entertainment', 'Gift Shop', 'Gourmet Shop', 'Grocery Store',
             'Halal Restaurant', 'Harbor / Marina', 'Hostel', 'Hot Dog Joint', 'Hotel', 'Hotel Bar', 'Hotel Pool', 'Ice Cream Shop', 'Indian Restaurant', 'Indie Theater', 'Irish Pub', 'Italian Restaurant', 'Japanese Restaurant', 'Jazz Club',
             'Juice Bar', 'Karaoke Bar', 'Liquor Store', 'Lounge', 'Mediterranean Restaurant', "Men's Store", 'Mexican Restaurant', 'Middle Eastern Restaurant', 'Miscellaneous Shop', 
             'Museum', 'Music Store', 'Music Venue', 'National Park', 'New American Restaurant', 'Nightclub', 'Nightlife Spot', 'Other Great Outdoors', 'Outdoors & Recreation', 'Park', 'Performing Arts Venue',
             'Pier', 'Pizza Place', 'Playground', 'Plaza', 'Pool', 'Pool Hall', 'Poutine Place', 'Pub', 'Public Art', 'Racetrack', 
             'Recreation Center', 'Rental Car Location', 'Residential Building (Apartment / Condo)', 'Restaurant','Rock Club', 'Salad Place', 'Sandwich Place', 'Scenic Lookout', 'Seafood Restaurant',
             'Smoothie Shop', 'Soup Place', 'Southern / Soul Food Restaurant', 'Spa', 'Spanish Restaurant', 'Speakeasy', 'Sports Bar', 'Stables', 'Steakhouse', 
             'Sushi Restaurant', 'Taco Place', 'Tapas Restaurant', 'Tennis Court', 'Tex-Mex Restaurant', 'Thai Restaurant', 'Theater', 'Tourist Information Center', 
             'Travel & Transport', 'Vegetarian / Vegan Restaurant', 'Vietnamese Restaurant','Wine Bar', 'Wine Shop', 'Winery', 'Wings Joint' ]

In [90]:
NO_grouped = NO_grouped[venue_list]
NO_grouped.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,...,Thai Restaurant,Theater,Tourist Information Center,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint
0,ALGIERS POINT,0.0,0.0,0.0,0.0,0.0,0.130435,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478
1,AUDUBON,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,B. W. COOPER,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,BAYOU ST. JOHN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,BLACK PEARL,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


## 
Put the top 5 venues of a neighborhood into a dataframe along with other fields we have previously filled

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NO_grouped['Neighborhood']

for ind in np.arange(NO_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NO_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Bar,Wings Joint,Scenic Lookout,New American Restaurant,Café,Music Venue,Music Store,Gift Shop,Park,Grocery Store
1,AUDUBON,Park,Plaza,Concert Hall,Playground,Coffee Shop,Theater,Outdoors & Recreation,Sandwich Place,Smoothie Shop,Wings Joint
2,B. W. COOPER,Recreation Center,Wings Joint,General Entertainment,Eastern European Restaurant,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food
3,BAYOU ST. JOHN,Sandwich Place,Playground,Other Great Outdoors,Park,Lounge,Food Truck,Food Court,Food & Drink Shop,Food,Wings Joint
4,BLACK PEARL,Farmers Market,Pool,Food Truck,Grocery Store,Wings Joint,General Entertainment,Ethiopian Restaurant,Fast Food Restaurant,Flea Market,Flower Shop


## 
Here is the employment of K Means Clustering to cluster the neighborhoods

We use K=5 clusters

In [93]:
kclusters = 5

NO_grouped_clustering = NO_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NO_grouped_clustering)
kmeans.labels_[0:] 

array([2, 2, 2, 2, 2, 2, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 4, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 1, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2])

In [94]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', [int(i) for i in kmeans.labels_])

NO_merged = NO_start

# merge starting NO dataframe and the sorted venues dataframe
NO_merged = pd.merge(NO_start, neighborhoods_venues_sorted, on="Neighborhood")

NO_merged.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,-90.026093,29.946085,2,Wings Joint,Gift Shop,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
1,ALGIERS POINT,-90.051606,29.952462,2,Bar,Wings Joint,Scenic Lookout,New American Restaurant,Café,Music Venue,Music Store,Gift Shop,Park,Grocery Store
2,WHITNEY,-90.042357,29.947200,2,Nightclub,Gift Shop,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
3,AUDUBON,-90.121450,29.932994,2,Park,Plaza,Concert Hall,Playground,Coffee Shop,Theater,Outdoors & Recreation,Sandwich Place,Smoothie Shop,Wings Joint
4,OLD AURORA,-90.000000,29.924440,1,Concert Hall,Playground,Wings Joint,Donut Shop,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food


In [95]:
# create map
map_clusters = folium.Map(location=[NO_lat, NO_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NO_merged['Latitude'], NO_merged['Longitude'], NO_merged['Neighborhood'], NO_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine the clusters 

### Cluster 0 - RED

In [96]:
NO_merged.loc[NO_merged['Cluster Labels'] == 0, NO_merged.columns[list(range(9))]]

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,ST. CLAUDE,-90.039268,29.971814,0,Plaza,Park,Wings Joint,Japanese Restaurant,Ethiopian Restaurant
15,CENTRAL CITY,-90.086689,29.939465,0,Park,Fried Chicken Joint,Seafood Restaurant,Cajun / Creole Restaurant,Grocery Store
25,MILAN,-90.098362,29.932548,0,Wine Shop,Dive Bar,Park,Theater,Wings Joint
44,LAKESHORE - LAKE VISTA,-90.097847,30.023253,0,Harbor / Marina,Park,Wings Joint,General Entertainment,Ethiopian Restaurant
51,PONTCHARTRAIN PARK,-90.039310,30.021989,0,Park,Wings Joint,General Entertainment,Ethiopian Restaurant,Farmers Market


### Cluster 1 - PURPLE

In [97]:
NO_merged.loc[NO_merged['Cluster Labels'] == 1, NO_merged.columns[list(range(9))]]

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,OLD AURORA,-90.000000,29.924440,1,Concert Hall,Playground,Wings Joint,Donut Shop,Ethiopian Restaurant
43,WEST END,-90.118618,30.013666,1,Playground,Wings Joint,General Entertainment,Ethiopian Restaurant,Farmers Market
48,ST. ANTHONY,-90.065746,30.015152,1,Playground,Wings Joint,General Entertainment,Ethiopian Restaurant,Farmers Market


### Cluster 2 - BLUE

In [98]:
print(NO_merged.loc[NO_merged['Cluster Labels'] == 2, NO_merged.columns[list(range(9))]].shape)
NO_merged.loc[NO_merged['Cluster Labels'] == 2, NO_merged.columns[list(range(9))]]

(56, 9)


,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,U.S. NAVAL BASE,-90.026093,29.946085,2,Wings Joint,Gift Shop,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant
1,ALGIERS POINT,-90.051606,29.952462,2,Bar,Wings Joint,Scenic Lookout,New American Restaurant,Café
2,WHITNEY,-90.042357,29.947200,2,Nightclub,Gift Shop,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant
3,AUDUBON,-90.121450,29.932994,2,Park,Plaza,Concert Hall,Playground,Coffee Shop
5,B. W. COOPER,-90.091753,29.951774,2,Recreation Center,Wings Joint,General Entertainment,Eastern European Restaurant,Ethiopian Restaurant
6,BAYOU ST. JOHN,-90.086517,29.976071,2,Sandwich Place,Playground,Other Great Outdoors,Park,Lounge
7,BLACK PEARL,-90.134883,29.935895,2,Farmers Market,Pool,Food Truck,Grocery Store,Wings Joint
8,BROADMOOR,-90.103812,29.946568,2,Recreation Center,Music Venue,Taco Place,Food Truck,Coffee Shop
9,MARLYVILLE - FONTAINEBLEAU,-90.113468,29.953001,2,Bar,Plaza,Food & Drink Shop,Lounge,Nightclub
10,GERT TOWN,-90.105829,29.960289,2,Asian Restaurant,Bakery,Seafood Restaurant,Wings Joint,Food & Drink Shop


### Cluster 3 - GREEN

In [99]:
NO_merged.loc[NO_merged['Cluster Labels'] == 3, NO_merged.columns[list(range(9))]]

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
56,FLORIDA DEV,-90.03253,29.981702,3,Wings Joint,Gift Shop,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant


### Cluster 4 - ORANGE

In [100]:
NO_merged.loc[NO_merged['Cluster Labels'] == 4, NO_merged.columns[list(range(9))]]

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
39,LAKEWOOD,-90.119991,29.985494,4,Baseball Field,Spa,Wings Joint,General Entertainment,Ethiopian Restaurant
41,CITY PARK,-90.093727,29.999172,4,Baseball Field,Park,Tennis Court,Wings Joint,Food & Drink Shop
